In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-11"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12802,2024-03-11,Brasil Nbb,20:00,Pinheiros,Unifacisa,2.13,1.66,159.5,1.81,1.89,1.5,1.97,1.51,WtCFufL4,0.469484,0.602410,0.552486,0.529101,0.071893,0.4,1.0,NaN,NaN,161.060,66.028501,0.409962,111.332,11.285864,0.101371,149.978,243.770,120.24,126.96,0.0,0.0,0.0,0.0,0.175377,0.030578,0.186936,-0.21,-0.042,-26.904762,0.556338,0.5,-0.056338,-3.68,-0.736,-0.896739,0.483805,0.2,-0.283805
12803,2024-03-11,Eua Nba,20:30,Cleveland Cavaliers,Phoenix Suns,1.81,2.07,225.5,1.82,1.94,-2.5,2.01,2.31,dAMhoQ40,0.552486,0.483092,0.549451,0.515464,0.035578,0.6,0.4,NaN,NaN,184.674,26.346300,0.142664,197.466,51.141188,0.258987,167.412,199.602,218.09,188.10,0.0,0.0,0.0,0.0,0.094767,0.045134,0.098209,-0.29,-0.058,-13.965517,0.621974,0.8,0.178026,-3.89,-0.778,-1.375321,0.581470,0.4,-0.181470
12804,2024-03-11,Eua Nba,21:00,Chicago Bulls,Dallas Mavericks,2.39,1.62,231.5,1.81,1.93,2.5,1.97,1.38,z1LdppK6,0.418410,0.617284,0.552486,0.518135,0.035694,0.8,0.4,NaN,NaN,340.760,104.074244,0.305418,212.836,41.123534,0.193217,304.866,222.232,248.32,232.56,0.0,0.0,0.0,0.0,0.271557,0.045376,0.249070,0.06,0.012,115.833333,0.529933,0.6,0.070067,0.13,0.026,23.846154,0.493359,0.5,0.006641
12805,2024-03-11,China Cba,08:35,Guangzhou,Shanghai,1.87,1.87,206.5,1.85,1.89,-2.5,2.04,2.10,l2CSYlpL,0.534759,0.534759,0.540541,0.529101,0.069519,0.6,0.6,NaN,NaN,217.588,105.473005,0.484737,223.504,135.822247,0.607695,227.392,265.442,222.36,450.47,0.0,0.0,0.0,0.0,0.000000,0.015125,0.020496,4.15,0.830,1.048193,0.562902,0.6,0.037098,2.27,0.454,1.916300,0.503784,0.5,-0.003784
12806,2024-03-11,Coreia Do Sul Kbl,07:00,LG Sakers,Suwon KT,1.71,2.10,164.5,1.87,1.79,-2.5,1.93,2.40,6yEgFVLR,0.584795,0.476190,0.534759,0.558659,0.060986,0.4,0.8,NaN,NaN,163.362,19.504345,0.119393,145.222,36.481108,0.251209,143.672,148.790,185.25,172.96,0.0,0.0,0.0,0.0,0.144762,0.030912,0.153506,-3.80,-0.760,-0.934211,0.698453,0.4,-0.298453,-2.39,-0.478,-2.301255,0.622040,0.5,-0.122040
12807,2024-03-11,Eua Ncaa,22:30,Northern Kentucky,Wisc. Milwaukee,1.87,1.95,150.5,1.80,1.95,-2.5,2.05,2.20,OE7BfZze,0.534759,0.512821,0.555556,0.512821,0.047580,0.6,0.4,NaN,NaN,137.400,40.877812,0.297510,197.044,84.668190,0.429692,121.220,196.980,129.75,190.00,0.0,0.0,0.0,0.0,0.029617,0.056569,0.049913,1.25,0.250,3.480000,0.661838,0.8,0.138162,2.03,0.406,2.339901,0.416113,0.5,0.083887
12808,2024-03-11,Israel Superliga,13:30,Hapoel Beer Sheva,Bnei Herzliya,1.99,1.77,167.5,1.87,1.79,1.5,1.85,1.61,SWoKurrF,0.502513,0.564972,0.534759,0.558659,0.067484,0.6,0.4,NaN,NaN,249.724,85.242919,0.341349,139.552,45.105088,0.323213,166.772,235.498,195.02,218.96,0.0,0.0,0.0,0.0,0.082747,0.030912,0.098096,-3.01,-0.602,-1.644518,0.000000,0.0,0.000000,3.10,0.620,1.241935,0.000000,0.0,0.000000
12809,2024-03-11,República Tcheca Nbl,15:00,Decin,Opava,2.01,1.73,166.5,1.80,1.86,1.5,1.86,1.60,lUW1LLU7,0.497512,0.578035,0.555556,0.537634,0.075547,0.2,0.4,NaN,NaN,162.264,83.798418,0.516433,132.972,40.553135,0.304975,169.488,235.530,99.36,131.37,0.0,0.0,0.0,0.0,0.105877,0.023184,0.106270,2.53,0.506,1.996047,0.000000,0.0,0.000000,0.48,0.096,7.604167,0.000000,0.0,0.000000
12810,2024-03-11,Romênia Divisão A,14:00,CSU Sibiu,Dinamo Bucharest,1.57,2.31,154.5,1.87,1.79,-4.5,1.98,2.24,6cRG4Qp0,0.636943,0.432900,0.534759,0.558659,0.069843,0.6,0.2,NaN,NaN,223.524,173.893348,0.777963,157.772,49.953189,0.316616,147.068,216.782,137.60,230.88,0.0,0.0,0.0,0.0,0.269721,0.030912,0.087132,1.03,0.206,2.76699

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:30,Eua Nba,Cleveland Cavaliers,Phoenix Suns,225.5,1.82,1.0000,Over
1,08:35,China Cba,Guangzhou,Shanghai,206.5,1.85,1.0000,Over
2,07:00,Coreia Do Sul Kbl,LG Sakers,Suwon KT,164.5,1.87,1.0000,Over
3,22:30,Eua Ncaa,Northern Kentucky,Wisc. Milwaukee,150.5,1.80,1.0000,Over
4,14:00,Romênia Divisão A,CSU Sibiu,Dinamo Bucharest,154.5,1.87,1.0000,Over
5,22:15,Uruguai Liga Uruguaia,Urupan,Goes,163.5,1.85,1.0000,Over
6,19:00,Eua Ncaa,Coll Of Charltn,Towson,139.5,1.91,1.0000,Over
7,22:30,Eua Ncaa,Nebraska O.,Denver,156.5,1.91,1.0000,Over
8,16:00,Israel Liga Leumit,Maccabi Haifa,Gilboa Galil,162.5,1.81,0.9949,Over
9,12:30,Rússia Premier League Feminina,Nika Siktivkar F,Nadezhda F,139.5,1.85,1.0000,Over
